In [1]:
import rasterio
import geopandas as gpd
import glob
import zipfile

In [2]:
gdf_scms = gpd.read_file('data/folhas-20-oumais-quadriculas-treinamento.gpkg').to_crs(5533)

In [3]:
gdf_quadriculas = gpd.read_file('data/quadriculas-32m-nas-areas-de-favela-atual.gpkg', layer='quadriculas-de-treinamento').to_crs(5533)

In [4]:
gdf_edificacoes = gpd.read_file('data/edificacoes-nas-quadriculas.gpkg')

In [5]:
gdf_scm_quadriculas = gpd.sjoin(gdf_scms, gdf_quadriculas, how='right')

In [6]:
gpd.sjoin(gdf_quadriculas, gdf_edificacoes, how='right')

,index_left,left,top,right,bottom,id,CLASSID,FEATID,DATA_CRIAC,ESCALA,ALTURA_EDI,COD_SETOR_,geometry
0,NaN,NaN,NaN,NaN,NaN,NaN,21.0,4875.0,2007-03-11,1:1000,5.632,034,"MULTIPOLYGON (((334195.394 7392885.796, 334197..."
1,NaN,NaN,NaN,NaN,NaN,NaN,21.0,4876.0,2007-03-11,1:1000,5.532,034,"MULTIPOLYGON (((334198.192 7392873.669, 334193..."
2,NaN,NaN,NaN,NaN,NaN,NaN,21.0,4878.0,2007-03-11,1:1000,7.072,034,"MULTIPOLYGON (((334181.198 7392874.087, 334177..."
3,NaN,NaN,NaN,NaN,NaN,NaN,21.0,4879.0,2007-03-11,1:1000,7.392,034,"MULTIPOLYGON (((334177.987 7392867.815, 334183..."
4,NaN,NaN,NaN,NaN,NaN,NaN,21.0,4675.0,2007-03-11,1:1000,7.190,034,"MULTIPOLYGON (((334165.920 7392841.071, 334162..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90031,NaN,NaN,NaN,NaN,NaN,NaN,21.0,2810892.0,2007-03-11,1:1.000,2.890,062,"MULTIPOLYGON (((340932.652 7397339.384, 340941..."
90032,NaN,NaN,NaN,NaN,NaN,NaN,21.0,2815655.0,2007-03-11,1:1000,4.620,033,"MULTIPOLYGON (((334015.626 7392801.673, 333996..."
90033,NaN,NaN,NaN,NaN,NaN,NaN,21.0,2815661.0,2007-03-11,1:1000,7.750,033,"MULTIPOLYGON (((334088.563 7392770.539, 334052..."
90034,NaN,NaN,NaN,NaN,NaN,NaN,21.0,2815662.0,2007-03-11,1:1000,3.119,033,"MULTIPOLYGON (((334029.463 7392827.406, 334029..."


In [7]:
from rasterio.windows import Window
from rasterio.transform import Affine
import rasterio.features
import math

In [8]:
folder = 'ortofotosMDC2004/treinamento'

# Interando sobre cada folha SCM
for scm in gdf_scms.qo2004_cod.to_list():
    
    print(scm)

    # DEscomprime o ZIP correspondente a folha SCM
    with zipfile.ZipFile(f'{folder}/1O_{scm}.zip', 'r') as zip_ref:
        zip_ref.extractall(folder)

    # Abre o raster SCM
    dataset = rasterio.open(glob.glob(f'{folder}/1O_{scm}*.JPG')[0])

    # Cria dataset de mascara das edificacoes
    gdf_edif = gpd.clip(gdf_edificacoes, gdf_scms.loc[gdf_scms.qo2004_cod == scm].geometry)
    edif = rasterio.features.rasterize(shapes=gdf_edif.unary_union, out_shape=dataset.shape, default_value=1, transform=dataset.transform, all_touched=True)

    profile = dataset.profile
    profile.update({
        'count':1,
        'driver':'GTiff',
        "compress":"LZW",
        'crs':'EPSG:5533'
    })
    profile.pop('photometric')

    with rasterio.open(f'ortofotosMDC2004/edificacoes-scm/{scm}-edificacoes-mask.tif', 'w', **profile) as dst:
        # Read the data from the window and write it to the output raster
        dst.write(edif, 1)

    edif = rasterio.open(f'ortofotosMDC2004/edificacoes-scm/{scm}-edificacoes-mask.tif')

    # Intera sobre cada quadricula da folha SCM Atual
    for row in gdf_scm_quadriculas.loc[gdf_scm_quadriculas.qo2004_cod == scm].itertuples():
        
        # Janela de 256x256
        xoff = (row.geometry.bounds[0] - dataset.bounds.left)/.125
        yoff = (row.geometry.bounds[3] - dataset.bounds.top)/.125
        
        window = Window(xoff, abs(yoff), 256, 256)
        # transform = dataset.window_transform(window)
        transform = Affine.translation(row.geometry.bounds[0], row.geometry.bounds[3]) * Affine.scale(.125, -.125)
        # transform = Affine(.125, 0., (xoff * 0.125) + dataset.bounds.left, 0., -.125, dataset.bounds.bottom + (yoff * 0.125))

        profile = dataset.profile
        profile.update({
            'height': 256,
            'width': 256,
            'crs':'EPSG:5533',
            'transform': transform})

        with rasterio.open(f'ortofotosMDC2004/licoes/{scm}-{row.Index}.JPG', 'w', **profile) as dst:
            # Read the data from the window and write it to the output raster
            dst.write(dataset.read(window=window))

        # Edificacoes na determinada janela
        # print(profile)
        profile.update({
            'count':1,
            'driver':'GTiff',
            "compress":"LZW"
        })
        profile.pop('photometric')
        profile.pop('interleave')
        with rasterio.open(f'ortofotosMDC2004/licoes/{scm}-{row.Index}-mask.tif', 'w', **profile) as dst:
            dst.write(edif.read(window=window))

    
    # break

2342-352
2346-242
3313-312
3443-352
2346-324
2344-261
2344-262
2344-324
2344-332
2342-413
2326-164
2344-333
2344-334
2344-352
3213-333
3213-364
3213-421
3336-142
3211-254
2346-212
2324-424
2344-361
2344-362
2344-411
2344-413
2344-422
2344-441
2344-442
3326-432
3213-423
3213-424
3211-411
2346-223
3335-454
3335-461
3335-462
3335-463
3336-113
3336-114
2442-424
2442-432
2442-452
2222-212
3311-354
2446-234
3211-414
3211-462
2442-461
3213-112
3213-123
4314-453
2344-162
2344-164
3313-151
3212-122
2346-163
3335-452
3336-141
3321-421
2342-234
2342-241
2342-242
2342-243
3315-343
2346-331
2346-333
2346-361
2346-413
2326-134
4312-222
4312-223
4312-232
3211-224
2344-223
4313-112
4313-114
3325-334
3334-412
3334-421
3334-422
3334-424
3336-232
3334-444
3334-452
3334-453
3334-461
3334-463
4312-241
3325-413
3325-421
3325-442
4311-164
4314-222
4314-223
4314-242
4315-113
3336-344
3336-351
3336-354
3341-124
3324-124
3324-151
3333-142
3336-353
2322-433
4313-353
3434-344
3434-353
4311-332
4311-334
3334-231
4